In [24]:
import os
import numpy as np
import pandas as pd

data = pd.read_json('/content/Dataset for Detection of Cyber-Trolls.json', lines= True)
data.head()

,content,annotation,extras
0,Get fucking real dude.,"{'notes': '', 'label': ['1']}",NaN
1,She is as dirty as they come and that crook ...,"{'notes': '', 'label': ['1']}",NaN
2,why did you fuck it up. I could do it all day...,"{'notes': '', 'label': ['1']}",NaN
3,Dude they dont finish enclosing the fucking s...,"{'notes': '', 'label': ['1']}",NaN
4,WTF are you talking about Men? No men thats n...,"{'notes': '', 'label': ['1']}",NaN


In [25]:
data['label'] = data['annotation'].apply(lambda x: x['label'][0])
data.drop(['extras'],axis = 1,inplace = True)
data.drop(['annotation'],axis = 1,inplace = True)


In [26]:
#preprocessing data: converting to lowercae, removing non-word chrs, removing digits
data['content'] = data['content'].apply(lambda x: x.lower() if isinstance(x, str) else x)
data['content'] = data['content'].str.replace("[^a-zA-Z#]", " ")
data['content'] = data['content'].replace(to_replace=r'[^\w\s]', value='', regex=True)
data['content'] = data['content'].replace(to_replace=r'\d', value='', regex=True)

In [27]:
X = data['content']
Y = data['label']


In [43]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
max_length = 100
embedding_dim = 100

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X)

# Convert texts to sequences of integers
X_seq = tokenizer.texts_to_sequences(X)

# Pad sequences to ensure uniform input size
X_pad = pad_sequences(X_seq, maxlen=max_length)

In [41]:
from sklearn.model_selection import train_test_split
Y = Y.astype('float32')
X_train, X_test, Y_train, Y_test = train_test_split(X_pad, Y, test_size=0.2, random_state=42)

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, 100, input_length=max_length),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test))


Epoch 1/10
500/500 [==============================] - 63s 119ms/step - loss: 0.5283 - accuracy: 0.7220 - val_loss: 0.4427 - val_accuracy: 0.7853
Epoch 2/10
472/500 [===========================>..] - ETA: 2s - loss: 0.2987 - accuracy: 0.8741